## Model Data using Cassandra

### The aim of the project is to solve the three queries given below.

### Introduction

There is a music streaming app called SoundCloud, that has been using their music streaming app and collecting data on songs and user activity and their aim is to analyze this data especially understanding what songs users are listening to. Currently, they are not making use of a NoSQL db and they have the data stored as a CSV file, thus its difficult for them to query the data. So our task is to create a NoSQL database for helping them with the analysis.


###  The image below is a screenshot of what the data appears like in the event_data.csv

<img src="event_data_image.jpg">

For Cassandra to get working, we need to make sure correct version of JAVA and python are installed. For that, I ran few commands in Command prompt which are shown in screeshorts below-->

<img src="Pic1.png">
<img src="Pic2.png">
<img src="Pic3.png">

Here, you can see I have successfully started my Apache Cassandra.

#### Import necessary Libraries

In [ ]:
pip install cassandra-driver

In [2]:
import pandas as pd
import numpy as np
import cassandra
import csv
from cassandra.cluster import Cluster

#### Creating a Cluster

In [3]:
# Connect to Cassandra
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

# Confirm connection
print("Cassandra Cluster connected.")

Cassandra Cluster connected.


#### Create & Set Keyspace

In [5]:
# Create a keyspace for SoundCloud data analysis
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS cloud_tracks 
    WITH REPLICATION = { 'class': 'SimpleStrategy', 'replication_factor': 1 }
""")

# Set the keyspace to use
session.set_keyspace('cloud_tracks')

# Confirm keyspace creation
print("Keyspace 'cloud_tracks' created and set.")

Keyspace 'cloud_tracks' created and set.


## List of Queries 

### Query1 Table1: 
Find the artist_name, song_title and length of song the SoundCloud app history that was heard during  session_number = 338, and item_in_session_number  = 4

In [8]:
# Create table for song history
session.execute("""
    CREATE TABLE IF NOT EXISTS song_history (
        session_number int,
        item_in_session_number int,
        artist_name text,
        song_title text,
        length float,
        PRIMARY KEY (session_number, item_in_session_number)
    )
""")
# Confirm table creation
print("Table 'song_history' created.")

Table 'song_history' created.


### Let's insert our data into of table

In [9]:
import csv

file_name = 'event_data.csv'

with open(file_name, encoding='utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader)  # Skip header
    for row in csv_reader:
        query = """
        INSERT INTO song_history (session_number, item_in_session_number, artist_name, song_title, length)
        VALUES (%s, %s, %s, %s, %s)
        """
        session.execute(query, (int(row[8]), int(row[3]), row[0], row[9], float(row[5])))

print("Data insertion completed.")

Data insertion completed.


### Validate our Data Model using a SELECT

In [14]:
# Retrieve song details from song_history
query = """
    SELECT artist_name, song_title, length 
    FROM song_history 
    WHERE session_number = 338 AND item_in_session_number = 4
"""

# Execute the query
rows = session.execute(query)

# Print the results
for row in rows:
    print(f"Artist -{row.artist_name}\nSong -{row.song_title}\nength -({row.length} seconds)")

Artist -Faithless
Song -Music Matters (Mark Knight Dub)
Length -(495.30731201171875 seconds)


### Query2 Table2: 
Find the artist_name, song_title (sorted by item_in_session_number) and name(fname and lname) of the user for user_id = 10, session_number = 182

In [15]:
# Create table for user sessions
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS user_session_songs (
        user_id INT,
        session_number INT,
        item_in_session_number INT,
        artist_name TEXT,
        song_title TEXT,
        first_name TEXT,
        last_name TEXT,
        PRIMARY KEY ((user_id, session_number), item_in_session_number)
    )
    """)
    
    print("Table 'user_session_songs' created.")

except Exception as e:
    print("Error creating the table:", e)                

Table 'user_session_songs' created.


### Let's insert our data into of table

In [16]:
import csv

file_name = 'event_data.csv'

with open(file_name, encoding='utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader)  # Skip header
    for row in csv_reader:
        # Prepare the INSERT statement
        query = """
        INSERT INTO user_session_songs (user_id, session_number, item_in_session_number, artist_name, song_title, first_name, last_name) 
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        
        # Match CSV columns to INSERT statement
        session.execute(query, (int(row[10]), int(row[8]), int(row[3]), row[0], row[9], row[1], row[4]))

print("Data inserted into the table successfully.")

Data inserted into the table successfully.


### Validate our Data Model using a SELECT

In [ ]:
pip install prettytable

In [22]:
# Query to find artist_name, song_title, and user names for specific user_id and session_number
query = """
    SELECT artist_name, song_title, first_name, last_name 
    FROM user_session_songs 
    WHERE user_id = 10 AND session_number = 182 
    ORDER BY item_in_session_number
"""

# Execute the query
rows = session.execute(query)

# Check if rows are returned
if rows:
    # Create a PrettyTable object for formatted output
    table = PrettyTable()

    # Define the table columns
    table.field_names = ["Artist Name", "Song Title", "First Name", "Last Name"]

    # Add rows to the table for each record retrieved
    for row in rows:
        table.add_row([row.artist_name, row.song_title, row.first_name, row.last_name])

    # Display the results in a table format
    print("\nResults for user_id = 10 and session_number = 182:")
    print(table)
else:
    print("No results found for user_id = 10 and session_number = 182.")



Results for user_id = 10 and session_number = 182:
+-------------------+------------------------------------------------------+------------+-----------+
|    Artist Name    |                      Song Title                      | First Name | Last Name |
+-------------------+------------------------------------------------------+------------+-----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+-------------------+------------------------------------------------------+------------+-----------+


### Query3 Table3: 
Find every name(fname and lname) of the user from the SoundCloud app history that listened to the song_title 'All Hands Against His Own'

In [29]:
 #Create a table for storing user names who listened to specific songs
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS song_listeners (
        song_title TEXT,
        user_id INT,
        first_name TEXT,
        last_name TEXT,
        PRIMARY KEY (song_title, user_id)
    )
    """)
    print("Table song_listeners created successfully")

except Exception as e:
    print("\n\nError occurred when creating the table",e)

Table song_listeners created successfully


### Let's insert our data into of table

In [30]:
file_name = 'event_data.csv'

with open(file_name, encoding='utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader)  # Skip header
    for row in csv_reader:
        query = """
        INSERT INTO song_listeners (song_title, user_id, first_name, last_name) 
        VALUES (%s, %s, %s, %s)
        """
        
        # Insert user data for each song they listened to
        session.execute(query, (row[9], int(row[10]), row[1], row[4]))

print("\nData inserted into the 'song_listeners' table successfully.")


Data inserted into the 'song_listeners' table successfully.


### Validate our Data Model using a SELECT

In [31]:
# Task: Query for users who listened to 'All Hands Against His Own'
query = """
    SELECT first_name, last_name 
    FROM song_listeners 
    WHERE song_title = 'All Hands Against His Own'
"""

# Execute the query
rows = session.execute(query)

# Create a PrettyTable object
table = PrettyTable()
table.field_names = ["First Name", "Last Name"]

# Add rows to the table
for row in rows:
    table.add_row([row.first_name, row.last_name])

# Display the table
print(table)

+------------+-----------+
| First Name | Last Name |
+------------+-----------+
| Jacqueline |   Lynch   |
|   Tegan    |   Levine  |
|    Sara    |  Johnson  |
+------------+-----------+


### Drop the tables before closing out the sessions

In [32]:
# Query to list tables in the current keyspace
tables_result = session.execute("SELECT table_name FROM system_schema.tables WHERE keyspace_name='cloud_tracks'")

print("Tables in keyspace 'cloud_tracks':")
for table in tables_result:
    print(table.table_name)

Tables in keyspace 'cloud_tracks':
song_history
song_listeners
user_session_songs


### Close the session and cluster connection¶

In [33]:
# Drop the tables
session.execute("DROP TABLE IF EXISTS song_history")
session.execute("DROP TABLE IF EXISTS user_session_songs")
session.execute("DROP TABLE IF EXISTS song_listeners")

In [34]:
# Close the session and cluster connection
session.shutdown()
cluster.shutdown()

# Print a validation message
print("Session and cluster connections have been closed successfully.")

Session and cluster connections have been closed successfully.
